In [4]:
%load_ext autoreload
%autoreload 2
import datetime
from glob import glob
import os
from pathlib import Path
import math
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from train_prophet import main_prophet
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [99]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths if path.count('DT') or path.count('day_no') or path.count('Amt')]
test_paths  = [path for path in test_paths  if path.count('DT') or path.count('day_no') or path.count('Amt')]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

In [147]:
START_DATE = '2017-11-25'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
train_df['datetime'] = train_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
test_df['datetime'] = test_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))

train_df['month'] = train_df['datetime'].map(lambda x: x.month)
test_df['month']  =  test_df['datetime'].map(lambda x: x.month)
train_df['date'] = train_df['datetime'].map(lambda x: x.date())
test_df['date']  =  test_df['datetime'].map(lambda x: x.date())

In [ ]:
train_df['day'] = train_df.TransactionDT / 3600 / 24
test_df['day']  = test_df.TransactionDT  / 3600 / 24
train_df['day_no'] = train_df['day'].map(lambda x: math.modf(math.floor(x*100)/100)[1])
train_df['hour']   = train_df['day'].map(lambda x: math.floor(math.modf(math.floor(x*100)/100)[0]*48))/2
train_df['time_zone'] = train_df['hour'].map(lambda x:
                                             '0.0-3.0' if (0<=x) and (x<=3)  else
                                             '3.5-6.0' if (3.5<=x) and (x<=6.0) else
                                             '6.5-10.0' if (3.5<=x) and (x<=10.0) else
                                             '10.5-17.0' if (10.5<=x) and (x<=17.0) else
                                             '17.5-21.5' if (17.5<=x) and (x<=21.5) else
                                             '22.0-23.5'
                                            )
train_df['month'] = train_df['day_no'].map(lambda x: 
                       12  if (1<=x) and (x<=31)    else
                       1  if (32<=x) and (x<=62)    else
                       2  if (63<=x) and (x<=90)    else
                       3  if (91<=x) and (x<=121)   else
                       4  if (122<=x) and (x<=151)  else
                       5  if (152<=x) and (x<=182)  else
                       6  if (183<=x) and (x<=212)  else
                       7  if (213<=x) and (x<=243)  else
                       8  if (244<=x) and (x<=274)  else
                       9  if (275<=x) and (x<=304)  else
                       10 if (305<=x) and (x<=335)  else
                       11 if (336<=x) and (x<=365)  else
                       12 if (366<=x) and (x<=396)  else
                       1
                      )

test_df['day_no'] = test_df['day'].map(lambda x: math.modf(math.floor(x*100)/100)[1])
test_df['hour']   = test_df['day'].map(lambda x: math.floor(math.modf(math.floor(x*100)/100)[0]*48))/2
test_df['time_zone'] = test_df['hour'].map(lambda x:
                                             '0.0-3.0' if (0<=x) and (x<=3)  else
                                             '3.5-6.0' if (3.5<=x) and (x<=6.0) else
                                             '6.5-10.0' if (3.5<=x) and (x<=10.0) else
                                             '10.5-17.0' if (10.5<=x) and (x<=17.0) else
                                             '17.5-21.5' if (17.5<=x) and (x<=21.5) else
                                             '22.0-23.5'
                                            )
test_df['month'] = test_df['day_no'].map(lambda x: 
                       12  if (1<=x) and (x<=31)    else
                       1  if (32<=x) and (x<=62)    else
                       2  if (63<=x) and (x<=90)    else
                       3  if (91<=x) and (x<=121)   else
                       4  if (122<=x) and (x<=151)  else
                       5  if (152<=x) and (x<=182)  else
                       6  if (183<=x) and (x<=212)  else
                       7  if (213<=x) and (x<=243)  else
                       8  if (244<=x) and (x<=274)  else
                       9  if (275<=x) and (x<=304)  else
                       10 if (305<=x) and (x<=335)  else
                       11 if (336<=x) and (x<=365)  else
                       12 if (366<=x) and (x<=396)  else
                       1
                      )

In [ ]:
for col in ['day_no', 'hour', 'time_zone', 'month', 'date', 'datetime']:
    feature = train_df[col].values
    to_pkl_gzip(obj=feature, path=f'../feature/raw_main/{col}_train')

for col in ['day_no', 'hour', 'time_zone', 'month', 'date', 'datetime']:
    feature = test_df[col].values
    to_pkl_gzip(obj=feature, path=f'../feature/raw_main/{col}_test')